In [1]:
from typing import Dict, Union
import sys
import os 
from pathlib import Path
import requests
import socket
import pandas as pd
import webbrowser
from requests_oauthlib import OAuth2Session
import gi
import tempfile

In [2]:
sys.path.append(str(Path().absolute().parent))

In [3]:
from eduvpn.oauth2 import one_request, get_open_port
from eduvpn.crypto import gen_code_challenge, gen_code_verifier, common_name_from_cert

# Settings

In [4]:
# client settings
DISCO_URI = 'https://disco.eduvpn.org/v2/'
ORGANISATION_URI = DISCO_URI + "organization_list.json"
SERVER_URI = DISCO_URI + "server_list.json"
client_id = "org.eduvpn.app.linux"
scope = ["config"]
code_challenge_method = "S256"

In [5]:
# our configuration
organisation = 'SURFnet bv'
LANGUAGE = 'nl'
COUNTRY = "nl-NL"

# Utils

In [6]:
def extract_translation(d: Union[str, Dict[str, str]]):
    if type(d) != dict:
        return d
    for m in [COUNTRY, LANGUAGE, 'en-US', 'en']:
        try:
            return d[m]
        except KeyError:
            continue
    return list(d.values())[0]  # otherwise just return first in list

# server list

In [7]:
servers_response = requests.get(SERVER_URI)
server_list = pd.DataFrame(servers_response.json()['server_list'])
server_list['display_name'] = server_list['display_name'].apply(extract_translation)
server_list

,server_type,base_url,public_key_list,country_code,support_contact,authentication_url_template,display_name,keyword_list
0,secure_internet,https://eduvpn.rash.al/,[Xv3l24gbMX8NtTnFQbWO2fGKPwKuc6EbjQDv8qw2GVk],AL,[mailto:helpdesk@rash.al],NaN,NaN,NaN
1,secure_internet,https://gdpt-eduvpndev1.tnd.aarnet.edu.au/,[HpY5RKF0OzYcYUcogKzgt1MvC6CxBmDJoUBsyiKjioA],AU,NaN,NaN,NaN,NaN
2,secure_internet,https://eduvpn.deic.dk/,[bRTz33KIuYo_w_-AbzNtdmLDqIm11_eGiHXQniojxY4],DK,[mailto:eduvpn@deic.dk],NaN,NaN,NaN
3,secure_internet,https://eduvpn.eenet.ee/,[jGpivOdwCRoLlexYKQjulZPPP4s3d9SVBFslI6RroAo],EE,[mailto:eduvpn@eenet.ee],NaN,NaN,NaN
4,secure_internet,https://eduvpn1.funet.fi/,[H4NTnM18BJgU_B3r8OBDVblBSfozB2Zu97I_ag2whmM],FI,[mailto:eduvpn@csc.fi],https://eduvpn1.funet.fi/Shibboleth.sso/Login?...,NaN,NaN
...,...,...,...,...,...,...,...,...
60,institute_access,https://eduvpn.univ-rennes1.fr/,NaN,NaN,[https://assistance.univ-rennes1.fr/],NaN,Université de Rennes 1,NaN
61,institute_access,https://hku.eduvpn.nl/,NaN,NaN,NaN,NaN,Hogeschool voor de Kunsten Utrecht,hku
62,institute_access,https://eduvpn.vamk.fi/,NaN,NaN,[mailto:helpdesk@vamk.fi],NaN,VAMK - University of Applied Sciences,NaN
63,institute_access,https://vu.eduvpn.nl/,NaN,NaN,"[mailto:servicedesk.it@vu.nl, tel:+31205980000]",NaN,Vrije Universiteit,NaN


# organisation list

In [8]:
organisation_response = requests.get(ORGANISATION_URI)
organization_list = pd.DataFrame(organisation_response.json()['organization_list'])
organization_list['display_name'] = organization_list['display_name'].apply(extract_translation)
organization_list['keyword_list'] = organization_list['keyword_list'].apply(extract_translation)
organization_list

,display_name,org_id,secure_internet_home,keyword_list
0,Danish Language Council,http://idp.dsn.dk/adfs/services/trust,https://eduvpn.deic.dk/,NaN
1,Business Academy Aarhus,http://adfs.eaaa.dk/adfs/services/trust,https://eduvpn.deic.dk/,NaN
2,KMD DSPARE3,http://dans-support-04.dans-idp-dev01.northeur...,https://eduvpn.deic.dk/,NaN
3,KMD DSUP_PATCH1,https://dans-idp.kmd.dk:7080,https://eduvpn.deic.dk/,NaN
4,KMD DANSWRC2,http://dans-support-02.dans-dp-dev01.northeuro...,https://eduvpn.deic.dk/,NaN
...,...,...,...,...
732,Uganda Virus Research Institute (UVRI),https://idp.uvri.go.ug/shibboleth,https://eduvpn.renu.ac.ug/,NaN
733,Uganda Martyrs Hospital Lubaga,https://idp.lubagahospital.org/idp/shibboleth,https://eduvpn.renu.ac.ug/,NaN
734,African Center of Excellence in Bioinformatics...,https://login.ace.ac.ug/idp/shibboleth,https://eduvpn.renu.ac.ug/,NaN
735,Bishop Stuart University,https://idp.bsu.ac.ug/idp/shibboleth,https://eduvpn.renu.ac.ug/,NaN


# Secure internet

In [9]:
secure_internet_list = server_list[server_list['server_type'] == 'secure_internet'].drop(['server_type'], axis=1)
secure_internet_list

,base_url,public_key_list,country_code,support_contact,authentication_url_template,display_name,keyword_list
0,https://eduvpn.rash.al/,[Xv3l24gbMX8NtTnFQbWO2fGKPwKuc6EbjQDv8qw2GVk],AL,[mailto:helpdesk@rash.al],NaN,NaN,NaN
1,https://gdpt-eduvpndev1.tnd.aarnet.edu.au/,[HpY5RKF0OzYcYUcogKzgt1MvC6CxBmDJoUBsyiKjioA],AU,NaN,NaN,NaN,NaN
2,https://eduvpn.deic.dk/,[bRTz33KIuYo_w_-AbzNtdmLDqIm11_eGiHXQniojxY4],DK,[mailto:eduvpn@deic.dk],NaN,NaN,NaN
3,https://eduvpn.eenet.ee/,[jGpivOdwCRoLlexYKQjulZPPP4s3d9SVBFslI6RroAo],EE,[mailto:eduvpn@eenet.ee],NaN,NaN,NaN
4,https://eduvpn1.funet.fi/,[H4NTnM18BJgU_B3r8OBDVblBSfozB2Zu97I_ag2whmM],FI,[mailto:eduvpn@csc.fi],https://eduvpn1.funet.fi/Shibboleth.sso/Login?...,NaN,NaN
5,https://eduvpn-poc.renater.fr/,[ePVNzE15h0yS6Xf3s8nJWmc8V6FeFziA3TZr0uOacFg],FR,[https://assistance.renater.fr/],https://eduvpn-poc.renater.fr/Shibboleth.sso/L...,NaN,NaN
6,https://eduvpn1.eduvpn.de/,[QjJHMit3vhHwLKi-fu2dXXSxMxnkskFVS3hMwyCnWQs],DE,"[mailto:eduvpn@dfn.de, tel:+49308842999120]",https://eduvpn1.eduvpn.de/saml/login?ReturnTo=...,NaN,NaN
7,https://eduvpn.myren.net.my/,[2WsUQpiUofRZ9IWZ8qQaOeQG4-RMy981QguOEMg4-e4],MY,[mailto:helpdesk@myren.net.my],https://eduvpn.myren.net.my/Shibboleth.sso/Log...,NaN,NaN
8,https://eduvpn.marwan.ma/,[aX-El_yRPdcUDF5S2smQ-9U7BzB35_1RtFYSjbHfEz8],MA,"[mailto:eduvpn-support@marwan.ma, tel:+2127000...",https://eduvpn.marwan.ma/saml/login?ReturnTo=@...,NaN,NaN
9,https://guest.eduvpn.no/,[qOLCcqXWZm9nmjsrwiJQxxWD606vDEJ2MIcc85oJmnE],NO,[mailto:kontakt@uninett.no],NaN,NaN,NaN


# make selection

In [10]:
organisation_info = organization_list[organization_list['display_name'] == organisation]
organisation_info

,display_name,org_id,secure_internet_home,keyword_list
711,SURFnet bv,https://idp.surfnet.nl,https://nl.eduvpn.org/,SURFnet bv SURF konijn powered by


# Select base URL of target country
We could connect to any country but we select the one matching the secure_interner_home

In [11]:
secure_internet = secure_internet_list[secure_internet_list['base_url'] == organisation_info['secure_internet_home'].iloc[0]]
secure_internet_url = secure_internet['base_url'].iloc[0]

# or if you want to connect to Uganda

In [12]:
# secure_internet_url = "https://eduvpn.renu.ac.ug/"

# fetch info from the base_url

In [13]:
info_url = secure_internet_url + '.well-known/vpn-user-portal'
info = requests.get(info_url).json()['api']['http://eduvpn.org/api#2']

In [14]:
api_base_uri = info['api_base_uri']
token_endpoint = info['token_endpoint']
authorization_endpoint = info['authorization_endpoint']

In [15]:
port = get_open_port()
redirect_uri = f'http://127.0.0.1:{port}/callback'

In [16]:
oauth = OAuth2Session(client_id, redirect_uri=redirect_uri, auto_refresh_url=token_endpoint, scope=scope)

In [17]:
code_verifier = gen_code_verifier()
code_challenge = gen_code_challenge(code_verifier)
authorization_url, state = oauth.authorization_url(url=authorization_endpoint,
                                                   code_challenge_method=code_challenge_method,
                                                   code_challenge=code_challenge)

In [ ]:
webbrowser.open(authorization_url)
response = one_request(port, lets_connect=False)

In [ ]:
code = response['code'][0]
assert(state == response['state'][0])

In [ ]:
token = oauth.fetch_token(token_url=token_endpoint,
                          code=code,
                          code_verifier=code_verifier,
                          client_id=oauth.client_id,
                          include_client_id=True,
                          )

# profile list

In [ ]:
profile_list_response = oauth.get(api_base_uri + '/profile_list')

In [ ]:
profile_list_response.json()['profile_list']['data']

In [ ]:
response = oauth.post(api_base_uri + '/create_keypair')
keypair = response.json()['create_keypair']['data']
private_key = keypair['private_key']
certificate = keypair['certificate']
common_name = common_name_from_cert(certificate.encode('ascii'))

# profile config

In [ ]:
profile_id = 'internet'
response = oauth.get(api_base_uri + f'/profile_config?profile_id={profile_id}')
config = response.text

# check_certificate

In [ ]:
response = oauth.get(api_base_uri + f'/check_certificate?common_name={common_name}')
assert(response.json()['check_certificate']['data']['is_valid'])